<a href="https://colab.research.google.com/github/lolovlad/colab/blob/master/%D0%97%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Изменение типов данных

Данные бывают разных типов, не всегда подходящих для анализа. В этой теме вы узнаете, как изменять типы данных для удобства расчётов.

Задачи сегодняшнего занятия:

•	Переводить строку в форматы даты и времени;

•	Превращать строковые значения в числовые методами **to_numeric()** и **astype()**;

•	Соединять таблицы методом **merge()**;

•	Создавать сводные таблицы методом **pivot_table()**.


Задача: SEO-оптимизаторы поставили задачу "Определить, какие товарные категории и подкатегории на сайте представлены в поисковиках хуже всего". В ответ на просьбу направить таблицы с источниками трафика, товарными категориями и подкатегориями, разработчики прислали экселевский файл.

Ранее мы встречались только с форматом **.csv** и методом **read_csv()**. Для прочтения файлов **Excel** есть особый метод **read_excel()**. Он похож на **read_csv()**, но в отличие от него, **read_excel()** нужно два аргумента: строка с именем самого файла или пути к нему, и имя листа **sheet_name**.

**import pandas as pd**

**df = pd.read_excel('/datasets/Экселевский файл.xlsx', sheet_name='Самый первый лист')**

Можно вызвать **read_excel()** и без второго аргумента. Тогда будет взят первый по счёту лист.
Вооружившись методом **read_excel()**, прочитаем файл с таблицами от разработчиков.


## Задача

1. Методом **read_excel()** прочтите первый лист **'traffic_data'**, сохраните его в переменной **data**. Выведите на экран первые 5 строк листа методом **head()**. Прочитайте названия столбцов:

•	**subcategory_id** — идентификатор товарной подкатегории, или её зашифрованное название;

•	**source** — источник трафика. В нашем случае источников два: прямой заход на сайт в таблице обозначен как direct, переход из поисковых систем — **organic**;

•	**visits** — количество визитов на страницу подкатегории за отчётный период.



In [3]:
import pandas as pd

data = pd.read_excel("seo_data.xlsx")
data.head(10)

,subcategory_id,source,visits
0,cf2e61c7af,direct,501165.668858
1,0cd903d1cc,direct,126342.359505
2,ef35bc88a7,direct,95626.321402
3,6ff9f4014c,direct,75680.653002
4,72bc238e4d,direct,64435.934651
5,88c78ea685,direct,39638.202995
6,179bfacf4a,direct,31007.471245
7,b0fb8f259a,direct,30324.097324
8,3c67eb2d90,direct,25878.814329
9,24e6c815c6,direct,25276.373358


Пока в первых 5 строчках таблицы виден только один источник трафика direct. Проверьте, какие источники есть в столбце source. Выведите на экран список уникальных значений этого столбца.


In [4]:
print(data["source"].unique())

['direct' 'organic']


Методом **read_excel()** прочтите второй лист **'subcategory_ids'** и сохраните его в переменной **subcategory_dict**. Выведите на экран его первые 5 строк методом **head()**.
Обратите внимание на названия столбцов: к знакомым нам идентификаторам подкатегорий **(subcategory_id)** добавились идентификаторы категорий **(category_id)** и названия подкатегорий **(subcategory_name)**.
Всё это похоже на словарь, где идентификаторам подкатегорий из цифр и латинских букв соответствуют понятные нам названия. Поэтому датафрейм называется **subcategory_dict**.


In [5]:
subcategory_dict = pd.read_excel("seo_data.xlsx", sheet_name="subcategory_ids")
subcategory_dict.head(10)

,subcategory_id,subcategory_name,category_id
0,cf2e61c7af,Мобильные телефоны,09f279a643
1,0cd903d1cc,Ноутбуки,3509869a61
2,ef35bc88a7,Телевизоры,09f279a643
3,6ff9f4014c,Планшеты,3509869a61
4,72bc238e4d,Наушники и Bluetooth-гарнитуры,09f279a643
5,88c78ea685,Шины,65922fa39c
6,179bfacf4a,Умные часы и браслеты,09f279a643
7,b0fb8f259a,Мониторы,3509869a61
8,3c67eb2d90,Наручные часы,6074ad29ac
9,24e6c815c6,Лекарственные препараты и БАДы,10a493e03b


Наконец, методом **read_excel()** прочтём третий лист **category_ids** и сохраним его в переменной **category_dict**. Выведите на экран первые 5 строк листа методом **head()**.

In [6]:
category_dict = pd.read_excel("seo_data.xlsx", sheet_name="category_ids")
category_dict.head(10)

,category_id,category_name
0,09f279a643,Электроника
1,3509869a61,Компьютерная техника
2,65922fa39c,Авто
3,6074ad29ac,"Одежда, обувь и аксессуары"
4,10a493e03b,Товары для здоровья
5,5a0d8ac946,Бытовая техника
6,7dbac4c1a3,Детские товары
7,ae99947659,Спорт и отдых
8,655ccc3c49,Товары для красоты
9,394f20d8fa,Строительство и ремонт


# Перевод строковых значений в числа

Читая данные, Pandas автоматически переводит их в подходящий формат. У него порой не получается, тогда приходится измениять тип данных вручную.

Рассмотрим файл со списком транзакций (платежей за одну покупку). Для каждой категории (буква в столбце **'category'**) посчитаем сумму продаж (значения в столбце **'amount'**).

In [7]:
transactions = pd.read_excel('ids.xlsx')
transactions.head()


,id,category,amount
0,600748331392,C,17515.36
1,600748331404,B,10117.56
2,600748331412,B,18489.26
3,600748331430,B,6620.22
4,600748331447,C,7559.9


Пока всё распознано корректно. В столбце **'id'** стоят идентификаторы транзакций, в столбце **category** — категории проданных товаров, в **amount** — стоимость каждого товара. Методом **info()** установим, как Pandas определил типы столбцов.

In [8]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49 non-null     object
 1   category  49 non-null     object
 2   amount    49 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


А вот и подозрительное поведение: столбцы **'id'** и **'amount'** распознаны как строки — тип **object**. Это значит, что в столбцах есть нечисловые значения или пропуски, а они помешают нашим расчётам.

К примеру, попробуем вычислить сумму продаж:

In [9]:
print(transactions['amount'].sum())

TypeError: unsupported operand type(s) for +: 'float' and 'str'

Сумма не считается из-за наличия строковых значений в столбце **'amount'**.
Собрать все строки да сделать их числами! Для этого есть стандартный метод Pandas — **to_numeric()**. Он превращает значения столбца в числовой тип **float64** (вещественное число).


In [10]:
transactions['amount'] = pd.to_numeric(transactions['amount'])

ValueError: Unable to parse string "Null" at position 21


У метода **to_numeric()** есть параметр **errors**. От значений, принимаемых **errors**, зависят действия **to_numeric** при встрече с некорректным значением:

•	**'errors='raise'** — дефолтное поведение: при встрече с некорректным значением выдаётся ошибка, операция перевода в числа прерывается;

•	**errors='coerce'** — некорректные значения принудительно заменяются на NaN;

•	**errors='ignore'** — некорректные значения игнорируются, но остаются.


In [11]:
transactions['amount'] = pd.to_numeric(transactions['amount'], errors='coerce')

Методом **to_numeric()** мы не только превратим строки в числовой тип там, где это возможно, но и выясним, на каких значениях метод не работает.
Особенность метода **to_numeric()** в том, что при переводе все числа будут иметь тип данных **float**. Это подходит далеко не всем значениям. Поэтому в нужный тип значения переводят методом **astype()**. Например, аргумент **('int')** метода **astype()** означает, что значение нужно перевести в целое число:
![image.png](attachment:image.png)



## Задача   
1. Переведите значения столбца **'id'** (таблицы **ids**) из строк в числа методом **to_numeric()**.



In [14]:
transactions['id'] = pd.to_numeric(transactions['id'], errors="coerce")

2. Попробуйте ещё раз перевести значения столбца **'id'** из строк в числа методом **to_numeric()**, но уже со значением **'coerce'** параметра **errors**. Это заменит вызывающий ошибку Null на NaN.
Полученную таблицу выведите на экран методом **tail()**: так как ошибка возникает на последней строке, 5 значений с конца будет достаточно.

In [15]:
transactions.tail()

,id,category,amount
44,6.007483e+11,A,8958.23
45,6.007483e+11,C,12771.10
46,6.007483e+11,E,9630.56
47,6.007483e+11,Null,NaN
48,NaN,A,14327.22


3. Вызовите метод **info()** после преобразований ещё раз. Проверьте, повлияла ли работа **pd.to_numeric(transactions['id'], errors='coerce')** на тип значений в столбце **id**.

In [16]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        48 non-null     float64
 1   category  49 non-null     object 
 2   amount    45 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.3+ KB


4. Методом **to_numeric()** превратите значения столбца **amount** в числа, заменив некорректные значения на NaN. Проверьте результаты преобразований методом **info()**.

In [17]:
transactions['amount'] = pd.to_numeric(transactions['amount'], errors='coerce')

5. Посчитайте общую стоимость проданных товаров — сумму значений столбца **amount** в датафрейме **transactions**. Результат выведите на экран.

In [18]:
print(transactions['amount'].sum())

596107.1


6.
Рассчитайте сумму продаж для каждой категории. Для этого сгруппируйте датафрейм **transactions** по столбцу **category** и примените метод **sum()** к столбцу **amount**. Выведите результат на экран.

In [19]:
print(transactions.groupby("category")["amount"].sum())

category
A       133988.50
B       168617.89
C       173332.50
D        62630.82
E        57537.39
Null         0.00
Name: amount, dtype: float64


7. Вернитесь к таблице с трафиком  (датасет data). При попытке сгруппировать датафрейм 'data' по источнику трафика и посчитать сумму визитов, вместо ожидаемого результата получили странное склеивание идентификаторов подкатегорий. Но теперь  вы сможете найти сумму визитов.

Переведите значения столбца **visits** датафрейма data в числовой тип методом **to_numeric()**.

In [20]:
data["visits"] = pd.to_numeric(data["visits"], errors="coerce")

8. Узнайте, что не так со строкой 964. Вызовите знакомый из вводного курса метод **loc**.
Он позволяет обращаться к датафрейму по значению индекса, в нашем случае как раз по номеру строки.


In [21]:
print(data.loc[964,])

subcategory_id     total
source            direct
visits               NaN
Name: 964, dtype: object


9. Вычислите, сколько всего строк в датафрейме и выведите на экран в формате:

Количество строк:

Затем выведите на экран строки, которые имеют значение **'total'** в столбце **'subcategory_id'**. Изучите полученные результаты: сколько строк портят весь датасет?

In [26]:
count = data.shape[0]
count_new = data[data['subcategory_id'] == 'total']["subcategory_id"].count()
print(f"Количество строк: {count}")
print(f"Количество строк total: {count_new}")

Количество строк: 1930
Количество строк total: 2


10. Оставьте в датафрейме только строки, удовлетворяющие условию **data['subcategory_id'] != 'total'**. Проверьте результат: выведите на экран датафрейм с условием **data['subcategory_id'] == 'total'**.

In [27]:
data = data[data['subcategory_id'] != 'total']
data.head(10)

,subcategory_id,source,visits
0,cf2e61c7af,direct,501165.668858
1,0cd903d1cc,direct,126342.359505
2,ef35bc88a7,direct,95626.321402
3,6ff9f4014c,direct,75680.653002
4,72bc238e4d,direct,64435.934651
5,88c78ea685,direct,39638.202995
6,179bfacf4a,direct,31007.471245
7,b0fb8f259a,direct,30324.097324
8,3c67eb2d90,direct,25878.814329
9,24e6c815c6,direct,25276.373358


11. Измените тип данных в столбце **['visits']** на **integer**. Проверьте полученный результат, вызвав метод **info()**.

In [28]:
data["visits"].astype("int")

0       501165
1       126342
2        95626
3        75680
4        64435
         ...  
1924        15
1925        15
1926        13
1927         9
1928         4
Name: visits, Length: 1928, dtype: int64

12. Сгруппируйте данные по источникам и вычислите количество визитов для каждого источника. Выведите результат на экран.

In [32]:
group = data.groupby("source")["visits"].sum()
group.head(10)

source
direct     2.369468e+06
organic    5.687882e+06
Name: visits, dtype: float64

# Метод merge()

Вы получили первые выводы: на органический трафик приходится в два раза больше визитов пользователей. Однако исследование на этом не закончено — нужно изучить трафик для каждой категории товаров.

Данные хранят в Excel-таблице из нескольких листов. Как смотреть на категории, подкатегории и трафик в одном месте? Идея — склеить таблицы!

Объединим несколько таблиц в одну методом **merge()**.

Метод **merge()** применяют к таблице, к которой присоединяют другую. У метода следующие аргументы:

•	**right** — имя **DataFrame** или **Series**, присоединяемого к исходной таблице ("правая таблица")

•	**on** — название общего списка в двух соединяемых таблицах: по нему происходит слияние ("ключи")

•	**how** — тип объединения (outer, inner, left, right)
Объединим таблицы **data** и **subcategory_dict** со следующими условиями:

•	**data** — таблица, к которой будем присоединять другую таблицу

•	**subcategory_dict** — таблица, которую присоединяем к data

•	**'subcategory_id'** — общий столбец в двух таблицах, по нему будем объединять

•	**how='left'** — id таблицы **data** включены в итоговую таблицу **data_subcategory**


In [33]:
data_subcategory = data.merge(subcategory_dict, on='subcategory_id', how='left')
print(data_subcategory.head(10))


  subcategory_id  source         visits                subcategory_name  \
0     cf2e61c7af  direct  501165.668858              Мобильные телефоны   
1     0cd903d1cc  direct  126342.359505                        Ноутбуки   
2     ef35bc88a7  direct   95626.321402                      Телевизоры   
3     6ff9f4014c  direct   75680.653002                        Планшеты   
4     72bc238e4d  direct   64435.934651  Наушники и Bluetooth-гарнитуры   
5     88c78ea685  direct   39638.202995                            Шины   
6     179bfacf4a  direct   31007.471245           Умные часы и браслеты   
7     b0fb8f259a  direct   30324.097324                        Мониторы   
8     3c67eb2d90  direct   25878.814329                   Наручные часы   
9     24e6c815c6  direct   25276.373358  Лекарственные препараты и БАДы   

  category_id  
0  09f279a643  
1  3509869a61  
2  09f279a643  
3  3509869a61  
4  09f279a643  
5  65922fa39c  
6  09f279a643  
7  3509869a61  
8  6074ad29ac  
9  10a493e03b 

## Задача:

1. Объедините таблицы **data_subcategory** и **category_dict**. Результат сохраните в переменной **data_final** и выведите на экран первые 10 строк новой таблицы.

In [35]:
data_final = data_subcategory.merge(category_dict, on='category_id', how='left')
print(data_final.head(10))

  subcategory_id  source         visits                subcategory_name  \
0     cf2e61c7af  direct  501165.668858              Мобильные телефоны   
1     0cd903d1cc  direct  126342.359505                        Ноутбуки   
2     ef35bc88a7  direct   95626.321402                      Телевизоры   
3     6ff9f4014c  direct   75680.653002                        Планшеты   
4     72bc238e4d  direct   64435.934651  Наушники и Bluetooth-гарнитуры   
5     88c78ea685  direct   39638.202995                            Шины   
6     179bfacf4a  direct   31007.471245           Умные часы и браслеты   
7     b0fb8f259a  direct   30324.097324                        Мониторы   
8     3c67eb2d90  direct   25878.814329                   Наручные часы   
9     24e6c815c6  direct   25276.373358  Лекарственные препараты и БАДы   

  category_id               category_name  
0  09f279a643                 Электроника  
1  3509869a61        Компьютерная техника  
2  09f279a643                 Электроника 

# Сводные таблицы

Сводная таблица — ваш помощник для обобщения данных и их наглядного представления.
Перед вами таблица со статистикой продаж товаров в интернет-магазине. Известны название товара, дата продажи, количество, цена и стоимость доставки.
![image.png](attachment:image.png)

По сводным таблицам можно точно сказать, сколько товара было продано в определённый день. Умение строить сводные таблицы позволяет быстро анализировать данные и сразу подготовить отчёты.

Например, по данной сводной таблице можно легко ответить  на вопрос: «Сколько товара продал интернет-магазин 4 июня 2019 года?».

![image.png](attachment:image.png)

В Pandas для подготовки сводных таблиц вызывают метод **pivot_table()**.

Аргументы метода:

•	**index** — столбец или столбцы, по которым группируют данные (название товара)

•	**columns** — столбец, по значениям которого происходит группировка (даты)

•	**values** — значения, по которым мы хотим увидеть сводную таблицу (количество проданного товара)

•	**aggfunc** — функция, применяемая к значениям (сумма товаров)

Построим сводную таблицу для задачи по SEO-оптимизации.
Сделаем по таблице **data_final** из предыдущего задания сводную таблицу методом **pivot_table()**:

**data_pivot = data_final.pivot_table(index=['category_name', 'subcategory_name'], columns='source',
                                    values='visits', aggfunc='sum')**


In [36]:
data_pivot = data_final.pivot_table(index=['category_name', 'subcategory_name'], columns='source', values='visits', aggfunc='sum')

Теперь вы видите название категории и её подкатегории со значениями объёма трафика по каждой.

Как видно, основная категория включает в себя подкатегорию и это представлено в структуре датафрейма: категория отображена иерархически главной над подкатегорией. Такие датафреймы содержат в себе мультииндекс. Часто при работе с такими датафреймами мультииндекс убирают, чтобы категория была отображена на каждой строчке датафрейма:

**data_pivot_with_reset_index = data_pivot.reset_index()
print(data_pivot_with_reset_index.head(10)) **



In [37]:
data_pivot_with_reset_index = data_pivot.reset_index()
print(data_pivot_with_reset_index.head(10))

source category_name            subcategory_name       direct       organic
0               Авто                Автоакустика  5915.606541  15433.405558
1               Авто               Автомагнитолы  7783.806626  18690.016622
2               Авто     Автомобильные инверторы   145.925199    150.592871
3               Авто   Автомобильные компрессоры   407.264303    790.164679
4               Авто    Автомобильные телевизоры   975.978827   2246.595618
5               Авто  Автомобильные холодильники   204.631014    285.110834
6               Авто            Автосигнализации   494.025691   1026.092391
7               Авто      Аккумуляторные батареи  1175.227510   2657.217881
8               Авто                   Антифризы   736.588757   1568.277621
9               Авто         Багажники, рейлинги   342.372257    664.308906


В таблице выше суммы визитов **visits** по каждому из источников **source (direct и organic)** представлены в отдельных столбцах. Такой вид таблицы называется «широкий». Он удобен, когда нужно сравнить значения столбцов. Хорошо заметно, например, что в подкатегории «Автомобильные инверторы» количество визитов из источников **direct** и **organic** практически одинаково, а в подкатегории «Автомагнитолы» из источника **organic** пришло почти в два раза больше посетителей, чем из **direct**.

Для группировки данных также подходит изученная вами ранее комбинация методов **groupby()** и **agg()** , но с ними таблица будет выглядеть иначе.

Метод **groupby()** принимает один аргумент — столбец (или список столбцов), по которым группируют данные. В метод **agg()** передают словарь. Его ключ — это названия столбцов, а значение — функции, которые будут к этим столбцам применены (например, **sum или count**). Такие функции называются агрегирующие.
Решим ту же задачу по SEO-оптимизации методами **groupby() и agg()**:


**data_grouped = data_final.groupby(['category_name','subcategory_name','source']).agg({'visits':'sum'})**

**print(data_grouped.head(10))**


In [38]:
data_grouped = data_final.groupby(['category_name','subcategory_name','source']).agg({'visits':'sum'})
print(data_grouped.head(10))

                                                       visits
category_name subcategory_name          source               
Авто          Автоакустика              direct    5915.606541
                                        organic  15433.405558
              Автомагнитолы             direct    7783.806626
                                        organic  18690.016622
              Автомобильные инверторы   direct     145.925199
                                        organic    150.592871
              Автомобильные компрессоры direct     407.264303
                                        organic    790.164679
              Автомобильные телевизоры  direct     975.978827
                                        organic   2246.595618


Сгруппировав данные методами **groupby()** и **agg()** вместо **pivot_table()**, мы получили точно такие же данные, но в несколько другом виде. Он называется «длинный». Методы **groupby()** и **agg()** удобны, когда нужно применить функцию к столбцу со сгруппированными визитами **visits** или создать новый столбец на его основе:

**data_grouped['daily_visits'] = data_grouped['visits'] / 30
print(data_grouped.head(10))**


In [39]:
data_grouped['daily_visits'] = data_grouped['visits'] / 30
print(data_grouped.head(10))

                                                       visits  daily_visits
category_name subcategory_name          source                             
Авто          Автоакустика              direct    5915.606541    197.186885
                                        organic  15433.405558    514.446852
              Автомагнитолы             direct    7783.806626    259.460221
                                        organic  18690.016622    623.000554
              Автомобильные инверторы   direct     145.925199      4.864173
                                        organic    150.592871      5.019762
              Автомобильные компрессоры direct     407.264303     13.575477
                                        organic    790.164679     26.338823
              Автомобильные телевизоры  direct     975.978827     32.532628
                                        organic   2246.595618     74.886521


## Задачи:

1. Самостоятельно создайте сводную таблицу для набора **data_final**. Результат сохраните в переменной **data_pivot** и выведите первых 10 значений на экран.

2. Создайте в таблице **data_pivot** новый столбец **'ratio'** и сохраните в нём значение отношения органического трафика **'organic'** к прямому **'direct'**. Выведите первые 10 строк таблицы на экран.

3. Отсортируйте таблицу по столбцу **'ratio'** в порядке убывания. Выведите первые 10 строк. Отсортированную таблицу сохранять не нужно.

4. Отсортируйте таблицу по столбцу **'ratio'** в порядке убывания и выведите последние 10 записей, где прямой трафик больше 1000. Выражение напишите в скобках функции **print()**, чтобы результат напечатался на экран.



In [ ]:
data_pivot = data_final.pivot_table(index=['category_name', 'subcategory_name'], columns='source', values='visits', aggfunc='sum').reset_index()
print(data_pivot.head(10))

In [ ]:
data_pivot["ratio"] = data_pivot["organic"] / data_pivot["direct"]
print(data_pivot.head(10))

In [ ]:
data_pivot = data_pivot.sort_values("ratio", ascending=False)
print(data_pivot.head(10))

In [52]:
t = data_pivot[data_pivot["direct"] > 1000]
print(t.nsmallest(n=10, columns=['ratio']))

source               category_name                  subcategory_name  \
424     Одежда, обувь и аксессуары                            Сапоги   
432     Одежда, обувь и аксессуары                             Туфли   
574         Строительство и ремонт        Нивелиры и лазерные уровни   
883                    Электроника          Защитные пленки и стекла   
768            Товары для здоровья                  Бандажи и ортезы   
525         Строительство и ремонт             Воздушные компрессоры   
538         Строительство и ремонт                     Дисковые пилы   
690                Товары для дома                            Люстры   
677                Товары для дома   Компьютерные и письменные столы   
88                 Бытовая техника  Оверлоки и распошивальные машины   

source       direct      organic     ratio  
424     1009.586105  2021.903456  2.002705  
432     1036.253838  2097.368209  2.023991  
574     1651.947395  3436.972506  2.080558  
883     1560.395673  3246.6